In [1]:
import pandas as pd
import sqlite3
sqlite_database = ("database.sqlite")
connection = sqlite3.connect(sqlite_database) #create connection with sqlite database

In [10]:
#retrieve data from Rate table, only limited to BusinessYear 2015, exclude Age as "family option" and individualrate 9999 9999.99
rate_data_2015 = pd.read_sql("""SELECT Distinct BusinessYear, StateCode, PlanId, RatingAreaId, Tobacco, Age, IndividualRate, IndividualTobaccoRate,
                        Couple, PrimarySubscriberAndOneDependent, PrimarySubscriberAndTwoDependents, PrimarySubscriberAndThreeOrMoreDependents,
                        CoupleAndOneDependent, CoupleAndTwoDependents, CoupleAndThreeOrMoreDependents
                        FROM Rate r
                        WHERE BusinessYear='2015' and Age !='Family Option' and IndividualRate !=9999  and IndividualRate !=9999.99;""", connection)

In [3]:
#retrive data from PlanAttributes table, only limisted to BusinessYear 2015
plan_data_2015 = pd.read_sql("""SELECT Distinct *
                        FROM PlanAttributes
                        WHERE BusinessYear='2015';""", connection)

In [13]:
#examine overview picture of rate data
print(rate_data_2015.shape)
print(rate_data_2015.info())
print(rate_data_2015.describe()) #please note we still have some high individualrate as 2084.7

(4560319, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560319 entries, 0 to 4560318
Data columns (total 15 columns):
BusinessYear                                 int64
StateCode                                    object
PlanId                                       object
RatingAreaId                                 object
Tobacco                                      object
Age                                          object
IndividualRate                               float64
IndividualTobaccoRate                        object
Couple                                       object
PrimarySubscriberAndOneDependent             object
PrimarySubscriberAndTwoDependents            object
PrimarySubscriberAndThreeOrMoreDependents    object
CoupleAndOneDependent                        object
CoupleAndTwoDependents                       object
CoupleAndThreeOrMoreDependents               object
dtypes: float64(1), int64(1), object(13)
memory usage: 521.9+ MB
None
       BusinessYear  In

In [14]:
#examine overview picture of rate data
print(plan_data_2015.shape)
print(plan_data_2015.info())
print(plan_data_2015.describe())

(31253, 176)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31253 entries, 0 to 31252
Columns: 176 entries, AVCalculatorOutputNumber to WellnessProgramOffered
dtypes: int64(9), object(167)
memory usage: 42.0+ MB
None
       BeginPrimaryCareCostSharingAfterNumberOfVisits  \
count                                    31253.000000   
mean                                         0.108310   
std                                          0.549437   
min                                          0.000000   
25%                                          0.000000   
50%                                          0.000000   
75%                                          0.000000   
max                                          4.000000   

       BeginPrimaryCareDeductibleCoinsuranceAfterNumberOfCopays  \
count                                       31253.000000          
mean                                            0.435734          
std                                             1.590785         

In [15]:
#PlanId in plan_data_2015 has format of 73836AK0650002-01, need to only keep the part before hyphen
plan_data_2015['PlanId']=plan_data_2015['PlanId'].str.split('-',expand=True)[0]

In [17]:
#only retrieve the PlanId and MetalLevel columns from plan_data_2015, and remove duplicate rows
plan_data=plan_data_2015[['PlanId','MetalLevel']]
plan_data_nodup=plan_data.drop_duplicates(keep='last')
print(plan_data_nodup.shape)
print(plan_data_nodup.head())

(10095, 2)
            PlanId MetalLevel
1   21989AK0030001       High
3   38344AK1020001       High
9   73836AK0710004     Silver
14  58670AK0010001       High
15  58670AK0010002        Low


In [18]:
#merge two dataframe (rate_data_2015 and plan_data_nodup) into one dataframe
df_final=pd.merge(rate_data_2015, plan_data_nodup, on='PlanId', how='left')

In [19]:
#display df_final
print(df_final.shape)
print(df_final['MetalLevel'].value_counts())

(4560319, 16)
Silver          1049582
Low              902399
High             883890
Gold             783656
Bronze           677258
Platinum         205804
Catastrophic      57730
Name: MetalLevel, dtype: int64


In [20]:
def cat_age(age):
    #this function will assign different group based on input age
    if age =='0-20':
        return 'Group A'
    elif age in str(range(21,45)):
        return 'Group B'
    elif age in str(range(45, 65)):
        return 'Group C'
    else:
        return 'Group D'


In [22]:
#add a new column called Age_Group, use cat_age function to assign age to different groups
df_final['Age_Group']=df_final['Age'].apply(cat_age)

0-20           100909
52              99098
35              99098
36              99098
37              99098
30              99098
31              99098
32              99098
33              99098
38              99098
39              99098
41              99098
53              99098
40              99098
43              99098
42              99098
45              99098
44              99098
47              99098
46              99098
49              99098
48              99098
34              99098
24              99098
25              99098
26              99098
50              99098
51              99098
56              99098
57              99098
54              99098
55              99098
58              99098
59              99098
64              99098
63              99098
62              99098
61              99098
60              99098
29              99098
28              99098
23              99098
22              99098
21              99098
27              99098
65 and ove

In [26]:
#review df_final dataframe
print(df_final['Age_Group'].value_counts())
print(df_final.info())

Group B    2378352
Group C    1981960
Group A     100909
Group D      99098
Name: Age_Group, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4560319 entries, 0 to 4560318
Data columns (total 17 columns):
BusinessYear                                 int64
StateCode                                    object
PlanId                                       object
RatingAreaId                                 object
Tobacco                                      object
Age                                          object
IndividualRate                               float64
IndividualTobaccoRate                        object
Couple                                       object
PrimarySubscriberAndOneDependent             object
PrimarySubscriberAndTwoDependents            object
PrimarySubscriberAndThreeOrMoreDependents    object
CoupleAndOneDependent                        object
CoupleAndTwoDependents                       object
CoupleAndThreeOrMoreDependents               object
M